In [1]:
#Import the important libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Read in the data
df = pd.read_csv("C:\\Users\\user\\Downloads\\CarPrice_Assignment.csv")
df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Usrs\\user\\Downloads\\CarPrice_Assignment.csv'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include = 'object')

In [ ]:
# car_ID is  randomly generated thereby not a predictive feature so we drop it
df = df.drop(columns = 'car_ID')

In [ ]:
df.head()

In [ ]:
df.nunique()

In [ ]:
#symboling is a categorical column so lets convert it 
df['symboling'] = df['symboling'].astype('object')
df.info()

In [ ]:
# Select the categorical and numerical columns
categorical_columns = df.select_dtypes("object")
numerical_columns = df.select_dtypes(['float64','int64',])

In [ ]:
categorical_columns.head()

In [ ]:
numerical_columns.head()

In [ ]:
#Lets observe how the numerical data is distributed
for columns in numerical_columns.columns:
    sns.distplot(df[columns])
    plt.show()

In [ ]:
for columns in numerical_columns.columns:
    sns.scatterplot(x = df[columns] ,y = df['price'])
    plt.show()

In [ ]:
#Observe the correlation between the features
plt.figure(figsize = (16,8))
sns.heatmap(numerical_columns.corr(),cmap = 'YlGnBu',annot = True);

In [ ]:
# Lets observe the unique items in each categorical features
for columns in categorical_columns.columns:
    print(df[columns].value_counts(),'\n\n') 

In [ ]:
#Clean the carName column
df['CarName'].value_counts()

In [ ]:
df['Car Brand'] = df['CarName'].apply(lambda x:x.split(' ')[0])

In [ ]:
df['Car Brand'].value_counts() # This look cleaner but at the end of the list,you will observe what might have been typos so we clean further

In [ ]:
df['Car Brand'].loc[(df['Car Brand'] == 'vokswagen') |(df['Car Brand'] == 'vw')] = 'volkswagen'

df['Car Brand'].loc[df['Car Brand'] == 'porcshce'] = 'porsche'

df['Car Brand'].loc[df['Car Brand'] == 'toyouta'] = 'toyota'

df['Car Brand'].loc[df['Car Brand'] == 'Nissan'] = 'nissan'

df['Car Brand'].loc[df['Car Brand'] == 'maxda'] = 'mazda'

df['Car Brand'].value_counts()

In [ ]:
categorical_columns.drop(columns = 'CarName',inplace = True)
categorical_columns.head()

In [ ]:
df['Car Brand']

In [ ]:
#merge the Car Brand Column to our categorical_column after dropping the Car Name column
categorical_columns = pd.merge(df['Car Brand'],categorical_columns,left_index = True,right_index = True)
categorical_columns.head()

In [ ]:
#Drp car Name column from df
df.drop(columns = 'CarName',inplace = True)

In [ ]:
df.head()

In [ ]:
#For the categorical columns,lets get dummies but drop those columns
df = pd.get_dummies(df,columns = categorical_columns.columns,drop_first = True)
df.head()

In [ ]:
#Remove the dependent variable from independent variables
X = df.drop(columns = 'price')
y = df['price']

In [ ]:
#Lets Scale our feautures to make coefficient interpretation easier
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X = pd.DataFrame(X_scaled,columns =X.columns)

In [ ]:
#Split the data into train and test
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 100)

In [ ]:
#Train your model
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
#Make predictions from your model
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

In [ ]:
#Evaluate your model

from sklearn.metrics import r2_score

train_set_accuracy = r2_score(y_true = y_train,y_pred = train_predictions)
test_set_accuracy = r2_score(y_true = y_test,y_pred = test_predictions)

print("Training Set R2 Score:",train_set_accuracy)
print("Test Set R2 Score:",test_set_accuracy)

In [ ]:
from sklearn.metrics import mean_squared_error

print("MSE for test set:",mean_squared_error(y_test,test_predictions))
print("MSE for training set:",mean_squared_error(y_train,train_predictions))

In [ ]:
#From our scores,we can see that our model is overfitting.Let us try to reduce the overfit by selecting optimal features from
# our many features and train the model with those features then pick one that results in better r2-scores but does not overfit

In [ ]:
from sklearn.feature_selection import RFE
import statsmodels.api as sm


train_r2 = []
test_r2 = []
train_MSE = []
test_MSE = []

for n_features in range(4,31):
    lg = LinearRegression()
    
    rfe = RFE(estimator = lg,n_features_to_select = n_features)
    
    rfe.fit(X_train,y_train)
    
    selected_features = X_train.columns[rfe.support_]
    
    X_train_rfe = X_train[selected_features]
    X_test_rfe = X_test[selected_features]
    
    #add a constant to the model
    X_train_rfe = sm.add_constant(X_train_rfe,has_constant = 'add')
    X_test_rfe = sm.add_constant(X_test_rfe,has_constant = 'add')
    
    #fit the model with the selected feaures
    rfe_model = sm.OLS(y_train,X_train_rfe).fit()
    
    #Make predictions
    test_predictions_rfe = rfe_model.predict(X_test_rfe)
    train_predictions_rfe = rfe_model.predict(X_train_rfe)
    
    #Evaluate your model
    train_r2.append(rfe_model.rsquared)
    test_r2.append(r2_score(y_test,test_predictions_rfe))
    
    train_MSE.append(mean_squared_error(train_predictions_rfe,y_train))
    test_MSE.append(mean_squared_error(test_predictions_rfe,y_test))
    

In [ ]:
#plotting R2 Score Against the number of features

plt.figure(figsize = (12,6))
plt.plot(range(4,31),train_r2,'r',label = 'R2 Train')
plt.plot(range(4,31),test_r2,'b',label = 'R2 Test')
plt.xticks(range(2,35))
plt.xlabel("No. of Features")
plt.ylabel("R2 Scores")
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
#Plotting Mean Squared error

plt.figure(figsize = (12,6))
plt.plot(range(4,31),train_MSE,'r',label = 'MSE Train')
plt.plot(range(4,31),test_MSE,'b',label = 'MSE Test')
plt.xticks(range(2,35))
plt.xlabel("No. of Features")
plt.ylabel("Mean Squared Error")
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
#11-13 features might be our best choice with no high bias and no high variance
#Lets make a model with 12 features
lg = LinearRegression()

rfe = RFE(estimator = lg,n_features_to_select = 13)
rfe.fit(X_train,y_train)

selected_columns = X_train.columns[rfe.support_]

X_train_rfe = X_train[selected_columns]
X_test_rfe = X_test[selected_columns]

X_train_rfe = sm.add_constant(X_train_rfe,has_constant = 'add')
X_test_rfe = sm.add_constant(X_test_rfe,has_constant = 'add')
    
rfe_model = sm.OLS(y_train,X_train_rfe).fit()

y_pred_train = rfe_model.predict(X_train_rfe)
y_pred_test = rfe_model.predict(X_test_rfe)
    
train_r2 = rfe_model.rsquared
test_r2 = r2_score(y_test,y_pred_test)
    
error_test=y_pred_test-y_test
error_train=y_pred_train-y_train
    
test_RMSE=(((error_test**2).mean())**0.5)
train_RMSE=(((error_train**2).mean())**0.5)
    
print("......................................R2_Score............................................")
print("Training data r2 Score:",train_r2)
print("Test Data r2 Score",test_r2)
print(".......................................RMSE................................................")
print("Training DataSet RMSE:",train_RMSE)
print("Testing Dataset RMSE",test_RMSE)
print(".......................................Rfe model summary.............................................\n\n")
print(rfe_model.summary())

In [ ]:
#Plot our Actual y points vs their predictions
plt.style.use('ggplot')
fig,ax = plt.subplots(figsize = (12,6))
sns.lineplot(x = y_test.index,y = y_test,label = 'True Value',ax = ax)
sns.lineplot(x = y_test.index,y = y_pred_test,label = 'Predicted Value',ax = ax)
ax.set_xlabel("Test examples index")
ax.set_ylabel("True and Predicted Values")
plt.show()


In [ ]:
#let observe the important features but eliminate the constant
key_features = rfe_model.params.index
key_features = key_features[1:]
key_features

In [ ]:
#Test the Linearity of the Model
#Lets use the plots of (observed vs predicted values) and (residual vs predicted values)

def linearity_test(model,y):
    predicted_vals = model.predict()
    residuals = model.resid
    
    sns.set_style("darkgrid")
    fig,ax = plt.subplots(1,2, figsize = (15,4))
    
    sns.regplot(x = predicted_vals,y = y,lowess = True,ax = ax[0],line_kws = {'color':'green'})
    ax[0].set_title("Predicted vs Observed Values",fontsize = 16)
    ax[0].set_xlabel("Predicted Values",fontsize = 13)
    ax[0].set_ylabel("True Y values(Observed)",fontsize = 13)
    
    sns.regplot(x = predicted_vals,y = residuals,lowess = True,ax = ax[1],line_kws = {'color':'blue'})
    ax[1].set_title("Predicted Values vs Residuals",fontsize = 16)
    ax[1].set_xlabel("Predicted Values",fontsize = 13)
    ax[1].set_ylabel("Residuals",fontsize = 13)
    fig.show()
    
linearity_test(rfe_model,y_train)

#Desired outcome is that points are symmetrically distributed around a diagonal line in the (Predicted vs Observed) plot and
# distributed a long an Howizontal line in the (Predicted vs Residuals) plot


In [ ]:
# We can observe that the Predicted vs Observed has most values closer to the diagonal
# Predicted vs Residuals does not have its values symmetrically distributed along a horizontal line
# but rather the values take a fan shaped pattern possibly because of outliers and therefore we cannot confirm LINEARITY.

#We can now test our model for homoscedasticity

In [ ]:
#Homoscedasticity  refer to the constant variance in the error terms against the predictions or against any independent variable
#This assumption can be tested by visual inspection of a standardized residual plot by the standardized regression 
#predicted value. Ideally, when the residuals are evenly scattered around the horizontal line, there is presence 
#of homoscedasticity; and when the residuals are not evenly scattered around the horizontal line and takes a various 
#shape like a bowtie, funnel shape, etc., then there is the presence of heteroscedasticity.


In [ ]:
import statsmodels.stats.api as sms

def homoscedasticity_test(model):
    
    predicted_values = model.predict()
    residuals = model.resid
    residuals_standardized = model.get_influence().resid_studentized_internal
    
    sns.set_style('darkgrid')
    
    fig,ax = plt.subplots(1,2,figsize = (15,8))
    sns.regplot(x = predicted_values,y = residuals,ax = ax[0],lowess = True,line_kws = {'color':'red'})
    ax[0].set_xlabel("Predicted Values",fontsize = 13)
    ax[0].set_ylabel("Residuals",fontsize = 13)
    ax[0].set_title("Predicted Values vs Residuals",fontsize = 16)
    
    sns.regplot(x = predicted_values,y = np.sqrt(np.abs(residuals_standardized)),ax = ax[1],lowess = True,line_kws = {'color' : 'red'})
    ax[1].set_xlabel("Predicted Values",fontsize = 13)
    ax[1].set_ylabel("Standardized residuals",fontsize = 13)
    ax[1].set_title("Predicted Values vs Standardized Residuals",fontsize = 16)
    fig.show()

In [ ]:
homoscedasticity_test(rfe_model)

In [ ]:
# The above graphs show that Homoscedasticity cannot be confirmed and again probably because of outliers
# We now test for the normality of residuals.If this assumption is violated,it causes problems with calculating confidence
# intervals.

In [ ]:
from scipy import stats

def normality_of_residuals_test(model):
    
    sm.ProbPlot(model.resid).qqplot(line = 's')
    plt.title("Q-Q Plot")
    
    jb = stats.jarque_bera(model.resid)
    sw = stats.shapiro(model.resid)
    ad = stats.anderson(model.resid, dist='norm')
    ks = stats.kstest(model.resid, 'norm')
    
    print(f'Jarque-Bera test ---- statistic: {jb[0]:.4f}, p-value: {jb[1]}')
    print(f'Shapiro-Wilk test ---- statistic: {sw[0]:.4f}, p-value: {sw[1]:.4f}')
    print(f'Kolmogorov-Smirnov test ---- statistic: {ks.statistic:.4f}, p-value: {ks.pvalue:.4f}')
    print(f'Anderson-Darling test ---- statistic: {ad.statistic:.4f}, 5% critical value: {ad.critical_values[2]:.4f}')
    print('If the returned Anderson Draling statistic is larger than the critical value, then for the 5% significance level, the null hypothesis that the data come from the Normal distribution should be rejected. ')
    
normality_of_residuals_test(rfe_model)

In [ ]:
#Q-Q Plot shows deviation from normal distirbution esp at tails and P-value in first 3 normality tests<0.05 and 
#Anderson-Darling statistic>AD critical value, thus null hypothesis that errors have normal dist is rejected
# Let us now identify outliers by ploting standardized residuals vs Leverage and cook's distance and remove them

In [ ]:
def outlier_detection(model,top_influencing_obs_count):
    influence = model.get_influence()

#leverage (hat values)
    leverage = influence.hat_matrix_diag

#When cases are outside of the Cook’s distance (meaning they have high Cook’s distance scores), 
#the cases are influential to the regression results. The regression results will be altered if we exclude those cases.
    cooks_d = influence.cooks_distance

#standardized residuals= (Residual/STD of Residuals)
    standardized_residuals = influence.resid_studentized_internal

#studentized residuals
    studentized_residuals = influence.resid_studentized_external 

    fig = plt.figure(figsize = (15,8))
    plt.scatter(leverage,standardized_residuals,alpha = 0.5)
    sns.regplot(leverage,standardized_residuals,scatter = False,ci = False,lowess = True,
               line_kws = {'color':'red',"lw":1,"alpha":0.8})
    fig.axes[0].set_xlim(0,max(leverage) + 0.01)
    fig.axes[0].set_ylim(-10,6)
    fig.axes[0].set_title("Standardized Residuals vs Leverage",fontsize = 16)
    fig.axes[0].set_xlabel("Leverage",fontsize = 13)
    fig.axes[0].set_ylabel("Standardized Residuals",fontsize = 13);
    
    leverage_top_n_obs = np.flip(np.argsort(cooks_d[0]), 0)[:top_influencing_obs_count]
    
    for i  in leverage_top_n_obs:
        fig.axes[0].annotate(i,xy = (leverage[i],studentized_residuals[i]))
        
    def graph(formula,x_range,label = None):
        x = x_range
        y = formula(x)
        plt.plot(x,y,label = label,lw = 1,ls = '--',color = 'red')
        
    p = len(rfe_model.params)
        
    graph(lambda x: np.sqrt((0.5 * p * (1 - x)) / x), np.linspace(0.001, max(leverage), 50),'Cook\'s distance')#cookd= 0.5 line
    plt.legend(loc='upper right'); 
    

In [ ]:
outlier_detection(model = rfe_model,top_influencing_obs_count=10)

In [ ]:
#We find that 2 observations 16 & 91 are outside cook's distance lines, thus they have to removed . 
#Also observation 24 is close to 0.5 cook's line and has a high standardized residual, also removing it 

X_train_no_outliers=X_train.drop(index=[16,24,91])
y_train_no_outliers=y_train.drop(index=[16,24,91])

In [ ]:
#Let us now Rebuild the model using RFE nad K Fold Cross Validation
from sklearn.feature_selection import RFE
import statsmodels.api as sm

train_r2 = []
test_r2 = []
train_RMSE = []
test_RMSE = []

for n_features in range(4,31):
    lg = LinearRegression()
    
    rfe = RFE(estimator = lg,n_features_to_select = n_features)
    
    rfe.fit(X_train_no_outliers,y_train_no_outliers)
    
    selected_features = X_train_no_outliers.columns[rfe.support_]
    
    X_train_rfe = X_train_no_outliers[selected_features]
    X_test_rfe = X_test[selected_features]
    
    X_train_rfe = sm.add_constant(X_train_rfe,has_constant = 'add')
    X_test_rfe = sm.add_constant(X_test_rfe,has_constant = 'add')
    
    rfe_model = sm.OLS(y_train_no_outliers,X_train_rfe).fit()
    
    y_pred_train = rfe_model.predict(X_train_rfe)
    y_pred_test = rfe_model.predict(X_test_rfe)
    
    #Metrics
    train_r2.append(rfe_model.rsquared)
    test_r2.append(r2_score(y_test,y_pred_test))
    
    error_test=y_pred_test-y_test
    error_train=y_pred_train-y_train_no_outliers
    
    test_RMSE.append(((error_test**2).mean())**0.5)
    train_RMSE.append(((error_train**2).mean())**0.5)

In [ ]:
#plotting R2 Score Against the number of features

plt.figure(figsize = (12,8))
plt.plot(range(4,31),train_r2,'r',label = 'R2 Train')
plt.plot(range(4,31),test_r2,'b',label = 'R2 Test')
plt.xticks(range(2,35))
plt.xlabel("No. of Features")
plt.ylabel("R2 Scores")
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
#Plotting Mean Squared error

plt.figure(figsize = (12,6))
plt.plot(range(4,31),train_MSE,'r',label = 'RMSE Train')
plt.plot(range(4,31),test_MSE,'b',label = 'RMSE Test')
plt.xticks(range(2,35))
plt.xlabel("No. of Features")
plt.ylabel("Root Mean Squared Error")
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
# Our Model appears to be highly overfitting so we can't select the optimal features

In [ ]:
# We can observe by what percentage the RMSE of the test set in higher compared to train set RMSE
RMSE_test_dividedby_train = [i / j for i, j in zip(test_RMSE, train_RMSE)]
RMSE_test_dividedby_train

In [ ]:
# We can cleary see that the model is overfitting.test RMSE is 32% higher than train RMSE for 4 features and 45% for 5 features.
# We can overcome this by trying K Fold cross validation to obtain optimal features

#Let us first prepare our data for K fold
X_cv = X.drop(index = [16,24,91])
y_cv = y.drop(index = [16,24,91])

In [ ]:
X_cv.reset_index(drop = True,inplace = True)
y_cv.reset_index(drop = True,inplace = True)

In [ ]:
print(X_cv.shape,y_cv.shape)

In [ ]:
#Perfom K-Fold Cross Validation

from sklearn.model_selection import KFold

kfold = KFold(n_splits = 5,shuffle = True,random_state = 42)

for n_features in range(5,31):
    train_r2 = []
    test_r2 = []
    train_RMSE = []
    test_RMSE = []
    
    for train,test in kfold.split(X_cv):
        lg = LinearRegression()
        
        rfe = RFE(estimator = lg,n_features_to_select = n_features)
        
        rfe.fit(X_cv.loc[train],y_cv[train])
        
        selected_columns = X_cv.columns[rfe.support_]
        
        X_cv_rfe = X_cv[selected_columns]
        
        X_cv_rfe = sm.add_constant(X_cv_rfe,has_constant ='add')
        
        rfe_model = sm.OLS(y_cv[train],X_cv_rfe.loc[train]).fit()
        
        y_pred_train = rfe_model.predict(X_cv_rfe.loc[train])
        y_pred_test = rfe_model.predict(X_cv_rfe.loc[test])
        
        #R-square
        train_r2.append(r2_score(y_pred_train , y_cv[train]))
        test_r2.append(r2_score(y_pred_test , y_cv[test]))
        
        #Error
        error_train = y_pred_train - y_cv[train]
        error_test = y_pred_test - y_cv[test]
        rmse_train=((error_train**2).mean())**0.5
        rmse_test=((error_test**2).mean())**0.5
        
        train_RMSE.append(rmse_train)
        test_RMSE.append(rmse_test)
        
        test_times_train=np.mean(test_RMSE)/np.mean(train_RMSE)
        # generate report
        print('n_features:{:1} |train_R2:{:2} |test_R2:{:3} |mean(rmse_train):{:4} |mean(rmse_test):{:5} |RMSE(test/train):{}'.
          format(n_features, round(np.mean(train_r2),4), round(np.mean(test_r2),4),
                 round(np.mean(train_RMSE),0),
                 round(np.mean(test_RMSE),0),round(test_times_train,2)))

In [ ]:
#Beyond 6 features,there is massive overfitting. To deal with this I ran the model with 5 and 6 features and found multicollinearity
# Features: carwidth and curbweight are highly correlated and latter has a VIF of 7 plus. 
#So when n=6, I remove 'curbweight' leaving us with n=5.
#So Model with 6 Features
import statsmodels.api as sm

lg = LinearRegression()

rfe = RFE(estimator = lg,n_features_to_select = 6)

rfe.fit(X_cv, y_cv)

selected_columns = X_cv.columns[rfe.support_]

X_cv_rfe = X_cv[selected_columns]

X_cv_rfe = sm.add_constant(X_cv_rfe,has_constant = 'add')

rfe_model = sm.OLS(y_cv,X_cv_rfe).fit()

rfe_model.summary()

In [ ]:
#WE now test multicollinearity-(none of the features should be highly correlated to other)

from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant


def variance_inflation_factors(X_df):
    X_df = add_constant(X_df)
    vifs = pd.Series(
        [1 / (1. - OLS(X_df[col].values, 
                       X_df.loc[:, X_df.columns != col].values).fit().rsquared) 
         for col in X_df],
        index=X_df.columns,
        name='VIF'
    )
    return vifs

In [ ]:
variance_inflation_factors(X_cv_rfe)

In [ ]:
#curbweight VIF=7.697143, highly correlated with carwidth (0.87 corr coeff) so removing curbweight
X_final =X_cv_rfe.loc[:,X_cv_rfe.columns !='curbweight']
X_final.head()

In [ ]:
#Fitting Linear Model Again
rfe_model =sm.OLS(y_cv,X_final).fit()

y_predictions=rfe_model.predict(X_final)


#Standard error/RMSE
error=y_predictions-y_cv

print('RMSE is: {}'.format(((error**2).mean())**0.5))

print(rfe_model.summary())

In [ ]:
fig, ax=plt.subplots(figsize=(15,8))
sns.lineplot(x=y_cv.index,y=y_cv,label='Actuals',color='blue',ax=ax)
sns.lineplot(x=y_cv.index,y=y_predictions,label='Predictions',color='red',ax=ax)
ax.set_title('Price: Actuals vs Predictions', fontsize=16)
ax.set_ylabel('Car Price',fontsize=13)

In [ ]:
#We now check for linearity for our new model

linearity_test(rfe_model,y_cv)

#Residuals more or less evenly scattered vs predicted values-looks fine

In [ ]:
homoscedasticity_test(rfe_model)  #both graphs show evenly spread residuals so homoscedasticity is present

In [ ]:
outlier_detection(model = rfe_model,top_influencing_obs_count=10) #no outliers

In [ ]:
normality_of_residuals_test(rfe_model)

In [ ]:
#The final model meets all the assumptions including no mutlicollinearity & no outliers and has an R-Square of 86%